# Detect unattended baggage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uclasystem/VQPy/blob/main/examples/unattended_baggage/demo.ipynb)

## Introduction

Unattended baggage could simply be an oversight of by the owner; or intentionally left behind, containing dangerous materials that poses a threat to public safety. Either case, detection is necessary.

In this example, we aim to identify baggages with no owner for some time period exceeding a set threshold. For simplicity, we say a baggage's owner in each video frame is:

- its owner from last frame, if that person is still present and within some distance from the baggage
- else, the person closest to the baggage and within some distance

## Environment setup

Python3.8 is recommended to avoid compatibility issues when installing YOLOX.

Please download video from [kaggle](https://www.kaggle.com/datasets/szahid405/baggage?select=baggage.mp4) and place it in the same directory as this notebook, aside from running the cell below.

In [ ]:
# install YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
# download YOLOX pretrained model
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth
!cd YOLOX && pip3 install .
# download VQPy, move vqpy/ to root directory for import
!git clone https://github.com/uclasystem/VQPy.git
!mv VQPy/vqpy ./
# install VQPy's dependencies
!pip3 install lap cython_bbox shapely
import vqpy
video_path = "./baggage.mp4"	# or change to wherever you put the video
save_folder = "./vqpy_outputs"
detector_model_dir = "./"

For reference, the working directory should look like:

```text
.
├── VQPy
├── YOLOX
├── baggage.mp4	# video to query on
├── vqpy	# make vqpy available for import
└── yolox_x.pth	# model checkpoint
```

## Unattended baggage query with VQPy

### Step 1: Define `VObj` types for baggage and person

Two types of VObjs are involved in the query: baggage and person. Querying on them requires creating the corresponding VObjs.

In [ ]:
class Person(vqpy.VObjBase):
    pass
class Baggage(vqpy.VObjBase):
    pass

#### Add property `owner` to baggage

We can add properties to VObjs by providing a function that computes its value. According to the simplified logic of finding a baggage's owner described above, we need the following to find the owner of the baggage:

- baggage's owner from last frame
	
	This requires property `owner` to be stateful, i.e. decorated with `@stateful`. We can then use `getv(owner, -2)` to access value of `owner` from last frame.
- identifier (track id) and coordinates of `Person`s in current frame

	To get properties of other VObjs, decorate the function with `@cross_vobj_property`. Argument `vobj_type` of the decorator takes the type of VObj to request properties from, and `vobj_input_fields` is the names of properties. All properties requested will be passed to the function in form of a list of tuples, each tuple containing properties of a VObj.

Adding property `owner`, we define `Baggage`:

In [ ]:
# helper function to calculate distance between two bounding boxes, in pixels
def distance(vobj1_tlbr, vobj2_tlbr):
    from math import sqrt

    center1 = (vobj1_tlbr[:2] + vobj1_tlbr[2:]) / 2
    center2 = (vobj2_tlbr[:2] + vobj2_tlbr[2:]) / 2
    # difference between center coordinate
    diff = center2 - center1
    return sqrt(diff[0] ** 2 + diff[1] ** 2)


class Baggage(vqpy.VObjBase):
    @vqpy.stateful()
    @vqpy.cross_vobj_property(vobj_type=Person, vobj_input_fields=("track_id", "tlbr"))
    def owner(self, person_ids_tlbrs):
        baggage_tlbr = self.getv("tlbr")
        prev_owner = self.getv("owner", -2)
        owner_id = None
        threshold = (
            baggage_tlbr[3] - baggage_tlbr[1]
        ) + 1  # set threshold to baggage's width
        min_dist = threshold + 1  # distance to nearest person
        # iterating through all people in the frame
        for person_id, person_tlbr in person_ids_tlbrs:
            dist = distance(baggage_tlbr, person_tlbr)
            if person_id == prev_owner and dist <= threshold:
                # return previous owner if still around
                return prev_owner
            if dist <= threshold and dist < min_dist:
                # update owner if closer
                owner_id = person_id
                min_dist = dist
        # new owner is returned (will return None if owner not found)
        return owner_id

### Step 2: Query on baggage's owner

To filter baggage that don't have any owner for some time period (i.e. `owner` is `None`), we use `vqpy.utils.continuing` with condition `lambda x: x is None` on property `"owner"`. Duration is set to 10 (seconds), and we use the name `no_owner` to describe the filter on `owner`.

In [ ]:
filter_cons = {
    "__class__": lambda x: x == Baggage,
    "owner": vqpy.utils.continuing(
        condition=lambda x: x is None, duration=10, name="no_owner"
    ),
}

For each baggage without owner, we output:

- `track_id`, tracking id, built-in property of VObj
- `tlbr`, bounding box, also built-in property of VObj. Use post-processing function to converted to string before serializing.
- `no_owner_periods`, a list of time periods (in seconds) of `Baggage` having no owner

	This property is added to `Baggage` by wrapper `vqpy.utils.continuing`, its name from `{name}_periods` where `name` "no_owner" is defined in the wrapper

In [ ]:
select_cons = {
    "track_id": None,
    "tlbr": lambda x: str(x),
    "no_owner_periods": None,
}

The query is:

In [ ]:
class FindUnattendedBaggage(vqpy.QueryBase):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        return vqpy.VObjConstraint(
            filter_cons=filter_cons,
            select_cons=select_cons,
            filename="unattended_baggage",
        )

### Step 3: Running the query

With the `Person` VObj and the query defined, we can run the query, where:

- `cls_name` is a tuple for mapping numerical outputs of object detector to str

	e.g. `vqpy.COCO_CLASSES` here starts with `("person", "bicycle", ...)`, meaning that we will map output `0` of object detector to COCO class `"person"`

- `cls_type` is a dictionary that maps name of object type (in str) to VObj types defined

	`{"person": Person, "backpack": Baggage, "suitcase": Baggage}` means we wish to map COCO class `person` to VObj type `Person`, and `backpack, suitcase`  to `Baggage`

- `tasks` is a list of queries to run on the video

In [ ]:
vqpy.launch(
    cls_name=vqpy.COCO_CLASSES,
    cls_type={"person": Person, "backpack": Baggage, "suitcase": Baggage},
    tasks=[FindUnattendedBaggage()],
    video_path=video_path,
    save_folder=save_folder,
    detector_model_dir=detector_model_dir,
)

## Expected result

Result of the query will be in `{save_folder}/{video_name}_{task_name}_{detector_name}.json`, output for this example should be in `./vqpy_outputs/baggage_unattended_baggage_yolox.json`.

One entry is created for each frame that has filter condition satisfied.

At 3102th frame (or 103s), we have output:

```json
{
  "frame_id": 3102,
  "data": [
    {
      "track_id": 522,
      "tlbr": "[ 65.91797 244.125   147.65625 397.6875 ]",
      "no_owner_periods": [
        [49, 83],
        [86, 103]
      ]
    }
  ]
}
```

The baggage we find is:

<img src="./demo.assets/frame3102.png" alt="with bounding box" style="zoom: 90%;" />